<a href="https://colab.research.google.com/github/jorgecif/CovidDisinformationDetection/blob/main/ModeloFinal_implementacion_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Detección de desinformación relacionada con COVID19** - implementacion

> Por: Jorge Orlando Cifuentes Cifuentes




Componentes principales:


*   Carga de modelos
*   Prueba



### Librerías y funciones

In [ ]:
# Generales
import warnings
warnings.filterwarnings('ignore')
#import csv
#import tensorflow as tf
import pandas as pd
import numpy as np
import os, joblib # Para exportar el modelo
from io import BytesIO
import requests
import time
from IPython.display import clear_output


# Gráficas
import matplotlib.pyplot as plt
import seaborn as sns


# Machine learning
from keras.layers import concatenate

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Activation, Flatten, Bidirectional


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import EarlyStopping


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

# Procesamiento lenguaje natural

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS, preprocess_string, strip_punctuation, strip_numeric
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(88)
import nltk
nltk.download('wordnet')




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Carga de modelos 

In [ ]:
# Modelos temática

mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_PrediccionTema/Tema_ML_vectorizer.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
vectorizer = joblib.load(mfile)

mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_PrediccionTema/Tema_ML_RF_model.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
rf_mod = joblib.load(mfile)

mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_PrediccionTema/Tema_ML_LOG_model.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
log_mod = joblib.load(mfile)

# Modelo LDA subtemática
mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_Subtema/model_LDA.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
model_lda_cargado = joblib.load(mfile)

mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_Subtema/dictionary.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
dictionary_cargado = joblib.load(mfile)



In [ ]:
# Modelo LSTM final
mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_FinalUnificado/tokenizer_base.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
tokenizer_base = joblib.load(mfile)

mLink = 'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_FinalUnificado/tokenizer.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
tokenizer = joblib.load(mfile)


In [ ]:
# Carga de modelos h5 desde github

import urllib.request

urllib.request.urlretrieve(
        'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_FinalUnificado/model_base_1input.h5?raw=true', 'model_base.h5')

urllib.request.urlretrieve(
        'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_FinalUnificado/model_LSTM_1input.h5?raw=true', 'model_lstm1.h5')

urllib.request.urlretrieve(
        'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_FinalUnificado/modelLSTM_2inputs.h5?raw=true', 'model_lstm2.h5')

urllib.request.urlretrieve(
        'https://github.com/jorgecif/CovidDisinformationDetection/blob/main/TrainedModels/Modelo_FinalUnificado/model_biLSTM_2inputs.h5?raw=true', 'model_lstm2_2.h5')

model_base = tf.keras.models.load_model('content/model_base.h5')


('model_lstm2_2.h5', <http.client.HTTPMessage at 0x7fdb2af7f470>)

In [ ]:
model_base = tf.keras.models.load_model('model_base.h5')
model_lstm1 = tf.keras.models.load_model('model_lstm1.h5')
model_lstm2 = tf.keras.models.load_model('model_lstm2.h5')
model_lstm2_2 = tf.keras.models.load_model('model_lstm2_2.h5')


In [ ]:
# Muestro algunos de los modelos cargados
print(vectorizer)
print(log_mod)
print(tokenizer)
print(model_base)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


#### Texto a revisar

In [ ]:
# Calculo predicciones de modelos con función

texto_a_revisar="You can protect yourself from COVID-19 by injecting, swallowing, bathing in or rubbing onto your body bleach, disinfectants or rubbing alcohols"
#texto_a_revisar="Bill Gates told us about the coronavirus in 2015"
#texto_a_revisar="Turmeric And Lemon Help Fight Against coronavirus."
#texto_a_revisar="Uganda is giving out 122GB of data to customers for free in response to COVID-19"
#texto_a_revisar="What we need to do to defeat the coronavirus is to consume more alkaline foods above the virus’ pH level"
#CANADA'S TRUDEAU SAYS WHOLE NATION FRUSTRATED BY SLOW PACE OF CORONAVIRUS VACCINATIONS
#WHO's Tedros 'very disappointed' China hasn't granted entry to coronavirus experts
#Zimbabwe daily COVID-19 cases jump to 1,365 as extended lockdown starts




### Aplico modelo para clasficar la temática

In [ ]:
# Función para aplicar modelo
def aplicar_modelo(datos_revisar, tokenizador, modelo):
  tokenizador=tokenizador
  modelo=modelo
  datos_revisar=datos_revisar
  linea_revisar_token=tokenizador.transform([datos_revisar])
  resultado=modelo.predict(linea_revisar_token)
  return resultado

In [ ]:
# Aplico modelo
pred_model_clasifica=aplicar_modelo(texto_a_revisar, vectorizer, log_mod)

In [ ]:
pred_model_clasifica

array([0])

In [ ]:
# Labels del tema
def get_keys(val,my_dict):
    for key, value in my_dict.items():
        if val == value:
            return key

tema_labels = {"Health":0, "Enviroment":1, "Lifestyle":2, "Finance":3, "Sports":4, "Politics":5, "Technology":6, "Science":7}
prediction_tema_label = get_keys(pred_model_clasifica,tema_labels)

In [ ]:
prediction_tema_label

'Health'

### Aplico modelo para extraer la subtemática y palabras claves

In [ ]:
# Convertir palabras plural en singular
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

In [ ]:
# Preprocesamiento

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
# Muestro los temas creados

for idx, topic in model_lda_cargado.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")



Topic: 0 
Words: 0.017*"test" + 0.013*"health" + 0.011*"australia" + 0.009*"travel" + 0.009*"clinic" + 0.008*"flight" + 0.008*"govern" + 0.008*"window" + 0.008*"hospit" + 0.008*"set"


Topic: 1 
Words: 0.022*"facebook" + 0.021*"post" + 0.021*"toilet" + 0.020*"paper" + 0.018*"show" + 0.017*"share" + 0.014*"novel" + 0.011*"lockdown" + 0.010*"thousand" + 0.010*"photo"


Topic: 2 
Words: 0.014*"infect" + 0.010*"pictur" + 0.009*"health" + 0.008*"spread" + 0.007*"case" + 0.007*"diseas" + 0.007*"mask" + 0.007*"countri" + 0.006*"report" + 0.006*"hand"


Topic: 3 
Words: 0.026*"case" + 0.016*"outbreak" + 0.012*"confirm" + 0.012*"state" + 0.012*"death" + 0.011*"trump" + 0.010*"health" + 0.010*"citi" + 0.009*"chines" + 0.008*"infect"


Topic: 4 
Words: 0.025*"health" + 0.018*"australia" + 0.014*"case" + 0.013*"test" + 0.013*"hospit" + 0.011*"public" + 0.009*"travel" + 0.008*"offic" + 0.007*"chief" + 0.007*"patient"




In [ ]:
# Recuento de palabras de topics
num_words=10

lda_topics_cargados = model_lda_cargado.show_topics(num_words=num_words)
topics_cargados = []
filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]

for topic in lda_topics_cargados:
    topics_cargados.append(preprocess_string(topic[1], filters))
topics_cargados

[['test',
  'health',
  'australia',
  'travel',
  'clinic',
  'flight',
  'govern',
  'window',
  'hospit',
  'set'],
 ['facebook',
  'post',
  'toilet',
  'paper',
  'show',
  'share',
  'novel',
  'lockdown',
  'thousand',
  'photo'],
 ['infect',
  'pictur',
  'health',
  'spread',
  'case',
  'diseas',
  'mask',
  'countri',
  'report',
  'hand'],
 ['case',
  'outbreak',
  'confirm',
  'state',
  'death',
  'trump',
  'health',
  'citi',
  'chines',
  'infect'],
 ['health',
  'australia',
  'case',
  'test',
  'hospit',
  'public',
  'travel',
  'offic',
  'chief',
  'patient']]

In [ ]:
# Función para aplicar modelo

def aplicar_lda(documento): 
    unseen_document=documento
    # Preprocesamiento
    bow_vector = dictionary_cargado.doc2bow(preprocess(unseen_document))
    # Aplico modelo
    prediction_lda=model_lda_cargado[bow_vector]

    probs=[]
    for i in range(0, len(prediction_lda)):
        probs.append(prediction_lda[i][1])
    max_probs=max(probs)

    position=[]
    for i in range(0,len(prediction_lda)):
        if max_probs==prediction_lda[i][1]:
            position=i
            break
    result=prediction_lda[position][0]
    
    return result

In [ ]:
pred=aplicar_lda(texto_a_revisar)
words_topics_lda=str(topics_cargados[pred])

In [ ]:
words_topics_lda

"['infect', 'pictur', 'health', 'spread', 'case', 'diseas', 'mask', 'countri', 'report', 'hand']"

In [ ]:
# Creo dataframe con columna adicional de predicción
datos_trabajo_pred=datos_trabajo
datos_trabajo_pred["pred_clasifica"]=pred_model_clasifica

#### Concateno resultados para que sirvan como entrada de modelo unificado

In [ ]:
# Datos de texto = INPUT 1
datos_analizar_texto=words_topics_lda+texto_a_revisar
input1=datos_analizar_texto

In [ ]:
# Datos numéricos = INPUT 2

metadatos=[np.asarray(pred_model_clasifica[0]).astype(np.int32),np.asarray(pred).astype(np.int32)]
metadatos=np.asarray(metadatos).astype(np.int32)
input2=[]
input2.append(metadatos)  


In [ ]:
input2[0]

array([0, 2], dtype=int32)

In [ ]:
input2

array([0, 2], dtype=int32)

### Aplico modelo unificado LSTM

In [ ]:
# Parámetros

# Tokenizador
tokenizer1input=tokenizer_base #Tokenizer para modelo sencillo
tokenizer2input=tokenizer #Tokenizer para modelo de 2 entradas

# Parámetros tokenizador
n_most_common_words = 10000
max_len = 300

# Modelos
# model_base = modelo base 
# model_lstm1 = modelo lstm 1 entrada
# model_lstm2 = modelo lstm 2 entradas
# model_lstm2_2 = modelo lstm bidireccional 2 entradas

# Entradas
texto_inicial=texto_a_revisar
# input1: entrada 1 texto original + palabras clave de subtema
# input2: entrada 2 metadatos tema, subtema



In [ ]:
# Predicción modelo base 1 entrada
test=texto_a_revisar

corpus_t=[]
corpus_t.append(test)
corpus_t=pd.Series(corpus_t)
  
sequences_reserva = tokenizer_base.texts_to_sequences(corpus_t.values)
transform_vect_reserva1= pad_sequences(sequences_reserva, maxlen=max_len)


In [ ]:
# Predicción
prediccion_base=model_base.predict(transform_vect_reserva1)
prediccion_base


array([[0.07467994]], dtype=float32)

In [ ]:
# Predicción modelo lstm 2 entradas
entrada1=input1
entrada2=input2
entrada2_arr=np.array(entrada2) # cambio a array para pasar al modelo

# Preparación
corpus_t=[]
corpus_t.append(entrada1)
corpus_t=pd.Series(corpus_t)
  
sequences_reserva = tokenizer.texts_to_sequences(corpus_t.values)
transform_vect_reserva2= pad_sequences(sequences_reserva, maxlen=max_len)


In [ ]:
# Predicción

#prediccion_lstm2=model_lstm2.predict([transform_vect_reserva2,input2])
#prediccion_lstm2=model_lstm2.predict(transform_vect_reserva2)
prediccion_lstm2=model_lstm2.predict({'nlp_input': transform_vect_reserva2, 'meta_input': entrada2_arr})
prediccion_lstm2

array([[1.3192422e-06]], dtype=float32)

In [ ]:
# Función para prueba de predicciones 1 entrada

def aplicar_modelo_unif_1input(a, modelo_probar, tokenizer):
    clf=modelo_probar
    tok=tokenizer
    # Tokenizacion
    corpus_1=[]
    corpus_1.append(a)
    corpus_2=pd.Series(corpus_1)
    sequences_reserva = tok.texts_to_sequences(corpus_2.values)
    transform_vect_reserva= pad_sequences(sequences_reserva, maxlen=max_len)
    prediccion=clf.predict(transform_vect_reserva)
    prediccion_a = [np.array(prediccion)]
    if prediccion > 0.5:
      label= "NO"
    else:
      label = "SI"
    alerta=[prediccion,label]
    return alerta

In [ ]:
# Prueba con función modelos 1 entrada

clasificar_texto=texto_a_revisar

resultado_prediccion=aplicar_modelo_unif_1input(clasificar_texto, model_base, tokenizer_base)

print("Texto: ", clasificar_texto)
print(" ")
print("Predicción: ")
print("Alerta de desinformación: ", resultado_prediccion[1]) 
print("Probabilidad asociada: ", resultado_prediccion[0][0])

Texto:  You can protect yourself from COVID-19 by injecting, swallowing, bathing in or rubbing onto your body bleach, disinfectants or rubbing alcohols
 
Predicción: 
Alerta de desinformación:  SI
Probabilidad asociada:  [0.07467994]


In [ ]:
# Función para prueba de predicciones 2 entradas

def aplicar_modelo_unif_2input(in1, in2, modelo_probar, tokenizer):
    clf=modelo_probar
    tok=tokenizer
    # Tokenizacion
    corpus_1=[]
    corpus_1.append(in1)
    corpus_2=pd.Series(corpus_1)
    sequences_reserva = tok.texts_to_sequences(corpus_2.values)
    in2_arr=np.array(in2) # cambio a array para pasar al modelo
    transform_vect_reserva= pad_sequences(sequences_reserva, maxlen=max_len)
    prediccion=clf.predict({'nlp_input': transform_vect_reserva, 'meta_input': in2_arr})    
    prediccion_a = [np.array(prediccion)]
    if prediccion > 0.5:
      label= "NO"
    else:
      label = "SI"
    alerta=[prediccion,label]
    return alerta

In [ ]:
# Aplico el modelo

in1=entrada1
in2=entrada2

resultado_prediccion_2=aplicar_modelo_unif_2input(in1, in2, model_lstm2, tokenizer)

print("Texto: ", clasificar_texto)
print(" ")
print("Predicción: ")
print("Alerta de desinformación: ", resultado_prediccion_2[1]) 
print("Probabilidad asociada: ", resultado_prediccion_2[0][0])

Texto:  You can protect yourself from COVID-19 by injecting, swallowing, bathing in or rubbing onto your body bleach, disinfectants or rubbing alcohols
 
Predicción: 
Alerta de desinformación:  SI
Probabilidad asociada:  [1.3192422e-06]
